In [ ]:
%pip install scikit-learn
%pip install tensorly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, img_as_float, transform
from scipy.ndimage import gaussian_filter, binary_opening, binary_closing
from sklearn.cluster import KMeans
import tensorly as tl
from tensorly.decomposition import tucker
import cv2 


image_path = 'C.jpg'
image = io.imread(image_path)
image = img_as_float(image)
image_lab = color.rgb2lab(image)  #  LAB color space

#  Adaptive Illumination Correction (CLAHE)**
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))  
image_lab[:,:,0] = clahe.apply((image_lab[:,:,0] * 255).astype(np.uint8)) / 255.0 

# Gaussian filtering
sigma_value = 1.5 
image_lab_filtered = np.zeros_like(image_lab)
for i in range(3):
    image_lab_filtered[:, :, i] = gaussian_filter(image_lab[:, :, i], sigma=sigma_value)

# third-order tensor
tensor_image = tl.tensor(image_lab_filtered)

# Tucker3 tensor decomposition
tucker_rank = [30, 30, 3] 
core, factors = tucker(tensor_image, rank=tucker_rank)

print("Core Tensor Shape:", core.shape)

core_resized = transform.resize(core, (image.shape[0], image.shape[1], 3), anti_aliasing=True)


core_resized_rgb = np.clip(color.lab2rgb(core_resized), 0, 1)


reconstructed_image = tl.tucker_to_tensor((core, factors))
reconstructed_image_rgb = np.clip(color.lab2rgb(reconstructed_image), 0, 1)

#features for clustering 
height, width, _ = image.shape
feature_vectors = reconstructed_image.reshape(height * width, -1)  

#K-means clustering 
n_clusters = 3  # Burn wound, healthy skin, background
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
kmeans.fit(feature_vectors)
labels = kmeans.labels_.reshape(height, width)


cluster_means = [np.mean(feature_vectors[labels.flatten() == i], axis=0) for i in range(n_clusters)]
burn_cluster = np.argmax([mean[1] for mean in cluster_means])  # LAB 'a' channel: higher values → red tones

#color-mapped segmentation image 
segmentation_map = np.zeros((height, width, 3))
colors = {burn_cluster: [1, 0, 0],  # Red for burn
           (burn_cluster + 1) % 3: [0, 1, 0],  # Green for healthy skin
           (burn_cluster + 2) % 3: [0, 0, 1]}  # Blue for background

for cluster, color in colors.items():
    segmentation_map[labels == cluster] = color

#Original and Segmented
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(segmentation_map)
plt.title('Segmented Image')
plt.axis('off')
plt.show()



